In [ ]:
import pandas as pd
import numpy as np
import pathlib
import sys

In [ ]:
raw_fold_drive = "G"

if sys.platform == 'darwin':
    print('mac')
    raw_fold = pathlib.Path('/users/ink/Google Drive/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/cottonwood/')
else:
    print('pc')
    raw_fold = pathlib.Path(f'{raw_fold_drive}:/Shared drives/UGS_Flux/Data_Processing/Jupyter_Notebooks/cottonwood/')



In [8]:
wellp = pd.read_excel('Milford_Pumping_2024_pi.xls', sheet_name='Sheet2')
wellp = wellp[wellp['WIN']>0]
wellp['WIN'] = wellp['WIN'].astype(int)
wellp

,WIN,SiteID,Inst_Flow_GPM,Horsepower,Site_kWh,Site_acft,SiteHours
0,19249,3.811001e+14,1800.0,150.0,1476.0,NaN,NaN
1,19248,3.811191e+14,1300.0,150.0,165017.0,378.1350,NaN
2,433364,3.811471e+14,NaN,125.0,2369.0,15.0970,NaN
3,19242,3.812571e+14,NaN,100.0,13223.0,17.4632,NaN
4,447765,3.813171e+14,900.0,30.0,59092.0,288.6240,NaN
...,...,...,...,...,...,...,...
65,19526,3.823241e+14,500.0,150.0,217225.0,31.1920,NaN
66,33180,3.823371e+14,320.0,30.0,83343.0,NaN,NaN
67,19445,3.824161e+14,790.0,75.0,120537.0,NaN,NaN
68,33465,3.824431e+14,1045.2,100.0,NaN,4.3210,2520.6


In [9]:
wlact = pd.read_csv('wlactivity.txt')
wlact = wlact.drop_duplicates(subset=['activityType', 'win'])
wlact.set_index(['win', 'activityType'], inplace=True)
wlact = wlact['ACTIV_COM_DATE'].unstack(level=1)
wlact_dict = wlact[['New','Abandon']].to_dict()

In [14]:
import pathlib

wlfeat = pd.read_csv('wlwellfeatures.txt',sep=',',
                     usecols=range(0,6))


wlfeat.set_index('win', inplace=True)
wlfeat['finishedDiameter'] = pd.to_numeric(wlfeat['finishedDiameter'], errors='coerce')
wlfeat['totalDepth'] = pd.to_numeric(wlfeat['totalDepth'], errors='coerce')
prop_dict = wlfeat[['finishedDiameter','totalDepth']].to_dict() 

In [11]:
wellp['finishedDiameter'] = wellp['WIN'].map(prop_dict['finishedDiameter'])
wellp['totalDepth'] = wellp['WIN'].map(prop_dict['totalDepth'])
wellp['NewWellDate'] = wellp['WIN'].map(wlact_dict['New'])
wellp['AbandonDate'] = wellp['WIN'].map(wlact_dict['Abandon'])
#wellp.to_csv('Milford_Pumping_2024_pi_enhanced.csv')

In [17]:
wellp.columns

Index(['well_id', 'SiteID', 'instflowgpm', 'horsepower', 'Site_kWh',
       'site_acft', 'sitehours', 'finishedDiameter', 'totalDepth',
       'NewWellDate', 'AbandonDate', 'proof_flow_gpm'],
      dtype='object')

In [18]:
rnm = {'WIN':'well_id',
       'Horsepower':'horsepower',
       'Inst_Flow_GPM':'instflowgpm',
       'SiteHours':'sitehours',
       'Site_acft':'site_acft',
       'Site_kWh':'total_kwh',}
wellp.rename(columns=rnm, inplace=True)
wellp['proof_flow_gpm'] = np.nan


In [20]:
clean = wellp[["well_id","horsepower","proof_flow_gpm","instflowgpm","sitehours","site_acft","total_kwh"]]

# Derive a flow to use for hours->volume (prefers instflow, else proof)
clean["flow_gpm_for_hours"] = np.where(clean["instflowgpm"].notna() & (clean["instflowgpm"]>0),
                                       clean["instflowgpm"],
                                       clean["proof_flow_gpm"])

# Convert gpm to acft/hour
clean["flow_acft_per_hour"] = clean["flow_gpm_for_hours"] * 60.0 / 325851.0

# Naive volume estimates (sanity checks, not final): 
# from hours and from energy (assuming mid prior ~350 kWh/acft)
assumed_kwh_per_acft = 350.0
clean["vol_from_hours_acft"] = clean["flow_acft_per_hour"] * clean["sitehours"]
clean["vol_from_energy_acft"] = clean["total_kwh"] / assumed_kwh_per_acft

# Basic profiling
profile = {
    "rows": len(clean),
    "missing": clean.isna().sum().to_dict(),
    "nonzero_counts": (clean.notna() & (clean!=0)).sum().to_dict(),
    "hp_summary": clean["horsepower"].describe().to_dict(),
    "site_acft_summary": clean["site_acft"].describe().to_dict(),
    "energy_summary_kwh": clean["total_kwh"].describe().to_dict(),
    "hours_summary": clean["sitehours"].describe().to_dict(),
    "instflow_summary": clean["instflowgpm"].describe().to_dict(),
    "proof_flow_summary": clean["proof_flow_gpm"].describe().to_dict(),
}

# Save the prepared data snapshot
prep_path = pathlib.Path("pumping_prepared.csv")
clean.to_csv(prep_path, index=False)

/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean["flow_gpm_for_hours"] = np.where(clean["instflowgpm"].notna() & (clean["instflowgpm"]>0),
/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean["flow_acft_per_hour"] = clean["flow_gpm_for_hours"] * 60.0 / 325851.0
/var/folders/jg/b66w1nm54nj99vk_9vx8vntr0000gn/T/ipykernel_25229/3566480235.py:14: S